In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 481, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 481 (delta 64), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (481/481), 8.88 MiB | 10.72 MiB/s, done.
Resolving deltas: 100% (249/249), done.


In [41]:
import pandas as pd
import numpy as np

X_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_X_train.csv")
X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_X_test.csv")
y_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_y_train.csv")


X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB


데이터 전처리
데이터 기본정보 확인

In [42]:
# 결측치를 채워주기 위해서 sklearn의 impute 사용
from sklearn.impute import SimpleImputer

# NaN으로 표시된 결측치를 mean으로 대체. strategy = constant로 하고 fill_value=0으로 하면 fillna()와 동일한 동작을 함.
imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')

# X_train['horsepower'] -> series X_train[['horsepower']] -> DataFrame
X_train[['horsepower']] = imputer.fit_transform(X_train[['horsepower']])
X_test[['horsepower']] = imputer.fit_transform(X_test[['horsepower']])

In [43]:
type(X_train['horsepower']), type(X_train[['horsepower']])

(pandas.core.series.Series, pandas.core.frame.DataFrame)

In [44]:
# 요약 통계량 확인으로 이상치 확인
X_train.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000
mean,23.732734,5.374101,189.994604,103.383212,2948.464029,15.580216,76.057554
std,7.647295,1.677084,105.471423,38.695458,862.949746,2.745907,3.605591
min,10.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,18.000000,4.000000,98.000000,75.000000,2206.250000,14.000000,73.000000
50%,23.000000,4.000000,140.500000,90.500000,2737.500000,15.500000,76.000000
75%,29.000000,6.000000,258.000000,118.750000,3560.000000,17.000000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [45]:
X_train.corr()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
mpg,1.000000,-0.767587,-0.791243,-0.761973,-0.832965,0.390410,0.549008
cylinders,-0.767587,1.000000,0.948018,0.841028,0.898443,-0.506298,-0.318203
displacement,-0.791243,0.948018,1.000000,0.898777,0.943124,-0.548478,-0.331432
horsepower,-0.761973,0.841028,0.898777,1.000000,0.875786,-0.690277,-0.384654
weight,-0.832965,0.898443,0.943124,0.875786,1.000000,-0.441569,-0.274378
acceleration,0.390410,-0.506298,-0.548478,-0.690277,-0.441569,1.000000,0.253719
model_year,0.549008,-0.318203,-0.331432,-0.384654,-0.274378,0.253719,1.000000


In [46]:
# isUSA는 차량모델의 미국산 여부
y_train

,isUSA
0,1
1,1
2,0
3,1
4,0
...,...
273,0
274,0
275,1
276,1


In [47]:
X_train['name'].value_counts()

ford pinto              5
toyota corolla          5
chevrolet chevette      4
ford maverick           4
pontiac catalina        3
                       ..
plymouth horizon tc3    1
mazda rx-4              1
ford mustang ii 2+2     1
ford fiesta             1
amc ambassador sst      1
Name: name, Length: 221, dtype: int64

In [48]:
X_train = X_train.iloc[:,1:]
X_test = X_test.iloc[:,1:]
# remove [name] column

In [49]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.3)

In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr)

X_tr = scaler.transform(X_tr)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [56]:
X_tr.shape, X_val.shape

((194, 7), (84, 7))

In [27]:
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())

from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth = 10)
modelDT.fit(X_tr, y_tr)

DecisionTreeClassifier(max_depth=10)

In [28]:
y_val_pred = modelKNN.predict(X_val)

y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

In [57]:
y_val_pred

array([1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1])

In [58]:
y_val_pred_probaKNN

array([[0. , 1. ],
       [0. , 1. ],
       [1. , 0. ],
       [0.8, 0.2],
       [0. , 1. ],
       [0. , 1. ],
       [0. , 1. ],
       [0.8, 0.2],
       [0.6, 0.4],
       [0. , 1. ],
       [0.6, 0.4],
       [0.4, 0.6],
       [0. , 1. ],
       [0. , 1. ],
       [0. , 1. ],
       [1. , 0. ],
       [0.2, 0.8],
       [0.6, 0.4],
       [0. , 1. ],
       [0.6, 0.4],
       [0.6, 0.4],
       [0.8, 0.2],
       [0. , 1. ],
       [0.6, 0.4],
       [0. , 1. ],
       [0. , 1. ],
       [0. , 1. ],
       [0.2, 0.8],
       [0.6, 0.4],
       [0.6, 0.4],
       [0. , 1. ],
       [1. , 0. ],
       [0. , 1. ],
       [0.8, 0.2],
       [0.2, 0.8],
       [0.2, 0.8],
       [0.6, 0.4],
       [0.2, 0.8],
       [0. , 1. ],
       [1. , 0. ],
       [0.6, 0.4],
       [0. , 1. ],
       [0. , 1. ],
       [0. , 1. ],
       [0.8, 0.2],
       [0. , 1. ],
       [0.8, 0.2],
       [0.6, 0.4],
       [0.8, 0.2],
       [0. , 1. ],
       [0. , 1. ],
       [0.4, 0.6],
       [0.8,

In [29]:
from sklearn.metrics import roc_auc_score
# pred_proba [:,0] ??
scoreKNN = roc_auc_score(y_val, y_val_pred_probaKNN[:,1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:,1])

print(scoreKNN, scoreDT)

0.8843149038461537 0.8545673076923077


In [59]:
best_model = None
best_score = 0

for i in range(2, 10):
  model = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
  model.fit(X_tr, y_tr.values.ravel())
  y_val_pred_proba = model.predict_proba(X_val)
  score = roc_auc_score(y_val, y_val_pred_proba[:,1])
  print(i,'개의 이웃확인 : ',score)
  if best_score <= score:
    best_model = model

2 개의 이웃확인 :  0.9180374361883153
3 개의 이웃확인 :  0.952070334656835
4 개의 이웃확인 :  0.9356211003970505
5 개의 이웃확인 :  0.9327850255246739
6 개의 이웃확인 :  0.9407260351673283
7 개의 이웃확인 :  0.9472490073737947
8 개의 이웃확인 :  0.9554736245036869
9 개의 이웃확인 :  0.9577424844015882


In [32]:
pred = best_model.predict_proba(X_test)[:,1]
print(pred)

[0.55555556 0.66666667 1.         1.         0.44444444 0.22222222
 0.44444444 0.33333333 0.22222222 0.55555556 1.         0.44444444
 1.         0.55555556 0.22222222 0.11111111 1.         1.
 0.11111111 1.         1.         1.         1.         1.
 1.         0.44444444 0.22222222 1.         1.         0.44444444
 0.22222222 0.         0.77777778 0.77777778 1.         0.55555556
 0.55555556 1.         0.11111111 0.77777778 1.         0.11111111
 0.11111111 0.44444444 0.44444444 0.22222222 0.33333333 1.
 0.55555556 1.         0.         0.22222222 1.         0.33333333
 1.         1.         1.         0.66666667 1.         0.33333333
 0.88888889 0.77777778 1.         1.         1.         1.
 1.         0.33333333 0.77777778 0.11111111 1.         1.
 0.44444444 0.         1.         0.33333333 1.         0.22222222
 0.33333333 1.         1.         1.         0.22222222 1.
 1.         0.88888889 0.         0.33333333 0.22222222 1.
 1.         1.         1.         1.         1.    

In [61]:
pd.DataFrame({'isUSA' : pred}).to_csv('경로')

',isUSA\n0,0.5555555555555556\n1,0.6666666666666666\n2,1.0\n3,1.0\n4,0.4444444444444444\n5,0.2222222222222222\n6,0.4444444444444444\n7,0.3333333333333333\n8,0.2222222222222222\n9,0.5555555555555556\n10,1.0\n11,0.4444444444444444\n12,1.0\n13,0.5555555555555556\n14,0.2222222222222222\n15,0.1111111111111111\n16,1.0\n17,1.0\n18,0.1111111111111111\n19,1.0\n20,1.0\n21,1.0\n22,1.0\n23,1.0\n24,1.0\n25,0.4444444444444444\n26,0.2222222222222222\n27,1.0\n28,1.0\n29,0.4444444444444444\n30,0.2222222222222222\n31,0.0\n32,0.7777777777777778\n33,0.7777777777777778\n34,1.0\n35,0.5555555555555556\n36,0.5555555555555556\n37,1.0\n38,0.1111111111111111\n39,0.7777777777777778\n40,1.0\n41,0.1111111111111111\n42,0.1111111111111111\n43,0.4444444444444444\n44,0.4444444444444444\n45,0.2222222222222222\n46,0.3333333333333333\n47,1.0\n48,0.5555555555555556\n49,1.0\n50,0.0\n51,0.2222222222222222\n52,1.0\n53,0.3333333333333333\n54,1.0\n55,1.0\n56,1.0\n57,0.6666666666666666\n58,1.0\n59,0.3333333333333333\n60,0.888888